In [1]:
import os
path = 'D:\Data_Science\StemAway'
os.chdir(path)

In [2]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import lxml.html as lh
import requests
import pandas as pd

In [3]:
url = 'https://community.smartthings.com/latest'

# Open connection crawing the page
html = requests.get(url).text

# Parse the web
soup = soup(html, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <title>
   Latest topics - SmartThings Community
  </title>
  <meta content="A community for everyone interesting in making their homes and lives smarter." name="description"/>
  <meta content="Discourse 2.5.0.beta5 - https://github.com/discourse/discourse version c8370b9c78df1161aa3db2699bbd53c3e04b9925" name="generator"/>
  <link href="https://aws1.discourse-cdn.com/smartthings/optimized/3X/7/1/716336ba8239a791818b9d1c5b2ef237659d45a3_2_32x32.png" rel="icon" type="image/png"/>
  <link href="https://aws1.discourse-cdn.com/smartthings/optimized/3X/6/0/60303ee32d0ca0a0624ac0b757ea61a67d895f34_2_180x180.png" rel="apple-touch-icon" type="image/png"/>
  <meta content="#ededf5" name="theme-color"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, user-scalable=yes, viewport-fit=cover" name="viewport"/>
  <link href="https://community.smartthings.com/latest" rel="canonical"/>
  <script type="app

In [4]:
#print(soup.title.text)
#print(soup.text)

In [5]:
#for link in soup.find_all("a"):
#   print("Inner Text: {}".format(link.text))
#   print("Title: {}".format(link.get("title")))
#   print("href: {}".format(link.get("href")))

In [7]:
def search(num_page):
    base = 'https://community.smartthings.com/latest?no_definitions=true&page='
    r = requests.get(base + str(num_page)).text
    bs = soup(r, 'lxml')
    
    items = soup.find_all("tr", {'class': 'topic-list-item'})
    topics = []
    categories = []
    tags = []
    urls = []
    for i in items:
        topic = i.td.a.text    
        category = i.td.div.find('span', {'class' : 'category-name'}).text      
        tag = i.td.div.div.text.replace('\n', '').strip()
        url = i.td.a.get('href')
        topics.append(topic)
        categories.append(category)
        tags.append(tag)
        urls.append(url)
    
    all_replies = soup.find_all("td", {'class': 'replies'})
    replies = []
    for r in all_replies:
        reply = r.text.replace('\n','').strip()
        replies.append(reply)
        
    views = []
    all_views = soup.find_all("td", {'class': 'views'})
    for v in all_views:
        view = v.text.replace('\n','').strip()
        views.append(view)
    
    dataset = pd.DataFrame({'topic': topics, 'category': categories, 'tag': tags, 'url': urls, 'reply': replies, 'views': views})
    return dataset

In [13]:
df = []
for page in range(10):
    d = search(page)
    df.append(d)
df = pd.concat(df).reset_index()
df.align(df, join = 'left')

df

,index,topic,category,tag,url,reply,views
0,0,Public calendar for SmartThings developer events,Groups & Events,,https://community.smartthings.com/t/public-cal...,14,8786
1,1,17 Amcrest IP cams w NVR to Blue Iris and S.T,Devices & Integrations,,https://community.smartthings.com/t/17-amcrest...,3,27
2,2,[RELEASE] Honeywell / Ademco Vista 20P Integra...,Community Created SmartApps,"honeywell , ...",https://community.smartthings.com/t/release-ho...,1258,66910
3,3,App Versus Website,Connected Things,afteriris,https://community.smartthings.com/t/app-versus...,11,1078
4,4,"Ecolink door sensor 4655BC0-R, has anybody use...",Devices & Integrations,,https://community.smartthings.com/t/ecolink-do...,3,32
...,...,...,...,...,...,...,...
295,25,'Fibaro Z-Wave FGK-101 Temperature & Door/Wind...,Community Created Device Types,"fibaro , ...",https://community.smartthings.com/t/fibaro-z-w...,522,30372
296,26,Globe Suite and ST Integration is Broken,Devices & Integrations,,https://community.smartthings.com/t/globe-suit...,346,14197
297,27,Integrating window shades with SmartThings (Ax...,Devices & Integrations,,https://community.smartthings.com/t/integratin...,148,20538
298,28,Withings Scales integration (May 2020),Devices & Integrations,,https://community.smartthings.com/t/withings-s...,11,123
